# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred_proba, threshold=0.5):
    # 확률을 기준으로 예측 레이블 생성
    y_pred = (y_pred_proba >= threshold).astype(int)  # 0.5 이상의 확률을 양성으로 간주

    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print("Confusion Matrix:\n", confusion)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

### 데이터 읽어오기


In [3]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("../../../data/train_data_0825.csv")
test_data = pd.read_csv("../../../data/test_data_0825.csv")

---

## 3. 모델 학습

### 모델 정의

optuna

한번더 옵튜나 돌려보았을때도 같은 성능나옴  

In [4]:
import optuna
from lightgbm import LGBMClassifier

# 'Normal'과 'AbNormal'을 숫자로 변환
train_data['target'] = train_data['target'].map({'Normal': 0, 'AbNormal': 1})

# 스레드홀드 설정
THRESHOLD = 0.3

def objectiveLGBM_dart(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 1300, 3300),
        'num_leaves': trial.suggest_int('num_leaves', 1800, 3800),
        'max_depth': trial.suggest_int('max_depth', 100, 450),
        'learning_rate': trial.suggest_float('learning_rate', 0.0003, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 50, 350),
        
        'boosting_type': 'dart',  # 'boosting'를 'boosting_type'으로 수정
        'random_state': RANDOM_STATE,
        'verbose': -1
    }
       
    model = LGBMClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data.drop("target", axis=1),
    train_data["target"],
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveLGBM_dart(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


[I 2024-08-25 19:46:04,466] A new study created in memory with name: no-name-a6a03a19-05b4-4d1e-8c48-7522551e22af
[I 2024-08-25 19:46:55,035] Trial 0 finished with value: 0.22253129346314326 and parameters: {'n_estimators': 1532, 'num_leaves': 3117, 'max_depth': 231, 'learning_rate': 0.18395093969213586, 'min_child_samples': 254}. Best is trial 0 with value: 0.22253129346314326.
[I 2024-08-25 19:49:36,348] Trial 1 finished with value: 0.22119815668202764 and parameters: {'n_estimators': 2665, 'num_leaves': 3710, 'max_depth': 323, 'learning_rate': 0.15402919924162803, 'min_child_samples': 75}. Best is trial 0 with value: 0.22253129346314326.
[I 2024-08-25 19:50:43,390] Trial 2 finished with value: 0.17770034843205573 and parameters: {'n_estimators': 2075, 'num_leaves': 2381, 'max_depth': 147, 'learning_rate': 0.0033286437432812747, 'min_child_samples': 174}. Best is trial 0 with value: 0.22253129346314326.
[I 2024-08-25 19:51:54,796] Trial 3 finished with value: 0.21921921921921922 and 

Best trial: score 0.2536443148688046, 
params {'n_estimators': 1483, 'num_leaves': 2909, 'max_depth': 436, 'learning_rate': 0.09776515732131329, 'min_child_samples': 321}


Best trial: score 0.2536443148688046,  
params {'n_estimators': 1483, 'num_leaves': 2909, 'max_depth': 436, 'learning_rate': 0.09776515732131329, 'min_child_samples': 321}  

.